In [1]:
import os
import argparse
import time

import matplotlib as mpl
mpl.use('TkAgg') # sets the backend for matplotlib
import matplotlib.pyplot as plt

from utils.redraw_obstacles import redraw_obstacles, set_non_blocking
from utils.create_draw_params import create_draw_params
from commonroad.common.file_reader import CommonRoadFileReader
from commonroad.visualization.draw_dispatch_cr import draw_object

import numpy as np

In [2]:
file_path = '/home/rong/VAE-Motion-Planning/scenarios/commonroad_data/highway-ramp.cr.xml'
scenario, _ = CommonRoadFileReader(file_path).open()

In [3]:
def show_lane(scenario):
    plt.figure(figsize=(250, 100))
    draw_params={'time_begin':0}
    handles = {}
    draw_object(scenario, handles=handles, draw_params=draw_params)
    plt.gca().autoscale()
    plt.gca().set_aspect('equal')
    plt.show()

In [4]:
show_lane(scenario)

KeyboardInterrupt: 

In [74]:
def plt_window(sce, dur, draw_params, size, pos):
    set_non_blocking()  # ensures interactive plotting is activated
    figsize = size
    fig = plt.figure(figsize=(figsize[0] / inch_in_cm, figsize[1] / inch_in_cm))
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    # ax.set_facecolor("black")
    # ax.set_cmap("binary")
    fig.add_axes(ax)
    fig.patch.set_facecolor('black')
    plt.xlim(pos[0] - size[0]/2, pos[0] + size[0]/2)
    plt.ylim( pos[1] -size[1]/2, pos[1] + size[1]/2)
    plt.set_cmap("plasma")
    plt.gca().set_aspect('equal')
    handles = {}  # collects handles of obstacle patches, plotted by matplotlib
    plot_limits = [pos[0] - size[0]/2, pos[0] + size[0]/2, pos[1] +-size[1]/2, pos[1] + size[1]/2]
    draw_params['time_begin'] = dur[0]
    draw_object(sce, handles=handles, draw_params=draw_params, plot_limits=plot_limits)
    # plt.gray()
    fig.canvas.draw()
    for handles_i in handles:
        if not handles_i:
            handles.pop()

    data = np.zeros(( (duration[1] - duration[0],) + fig.canvas.get_width_height()[::-1]), dtype='bool')

    t1 = time.time()
    # loop where obstacle positions are updated
    for i in range(dur[0], dur[1]):
        # ...
        # change positions of obstacles
        # ...
        draw_params['time_begin'] = i
        redraw_obstacles(sce, handles=handles, figure_handle=fig, plot_limits=plot_limits, draw_params=draw_params)
        buffer = np.frombuffer(fig.canvas.renderer.buffer_rgba(), dtype=np.uint8).reshape(fig.canvas.get_width_height()[::-1] + (4,))
        buffer = buffer[:, : , 0].astype(bool)
        data[(i-duration[0]), :, : ] = buffer
        print('fps:', (i - duration[0])/(time.time()-t1))
    return data

In [7]:
size = [200, 40]
pos = [350, 135]
duration = [0, 100]
inch_in_cm = 10
draw_params = create_draw_params()

origin_data = plt_window(sce=scenario, dur=duration, draw_params=draw_params, size=size, pos=pos)

fps: 0.0
fps: 13.50552868670346
fps: 18.756180616887836
fps: 22.48462268348513
fps: 24.515119191305185
fps: 25.69176526509481
fps: 26.87387366222361
fps: 27.57093945500679
fps: 28.17053558085765
fps: 28.92780833286715
fps: 29.37854247042046
fps: 29.71084289245975
fps: 29.88263331415635
fps: 30.23230200613673
fps: 30.27263606902873
fps: 30.184229918938836
fps: 29.938918397555952
fps: 30.24303974429013
fps: 30.341447966492275
fps: 30.57795458801067
fps: 30.58485094976434
fps: 29.744064383056493
fps: 28.783606748628348
fps: 28.40876896894851
fps: 28.632626643171957
fps: 28.8179901369092
fps: 29.02170588764418
fps: 29.065369627635125
fps: 29.187607349856833
fps: 29.367068754801565
fps: 29.412322126434834
fps: 29.6135791607156
fps: 29.525045678583794
fps: 29.051212373397362
fps: 28.302118238059062
fps: 28.420792390695134
fps: 28.515830498494182
fps: 28.501521961833966
fps: 28.01057833578202
fps: 27.750397863269395
fps: 27.621986205484056
fps: 27.56326024118709
fps: 27.00145628813445
fps: 26

In [8]:
def plt_data(data):
    fig = plt.figure()
    plt.gray()
    if len(data.shape) <=2:
        plt.imshow(data)
    else:
        myobj = plt.imshow(data[0, :, :])
        for row in data:
            myobj.set_data(row)
            plt.draw()
            plt.pause(0.01)

In [9]:
plt_data(origin_data)

In [10]:
from commonroad.scenario.scenario import Scenario

ego_scenario = Scenario(dt=0.1, benchmark_id=scenario.benchmark_id)
ego_scenario.lanelet_network = scenario.lanelet_network

In [93]:
ego_id = 114
# ego_obstacle = scenario.dynamic_obstacles[ego_id]
# ego_obstacle_new = copy.deepcopy(ego_obstacle)

In [94]:
obstacle = scenario.obstacle_by_id(ego_id)
scenario.remove_obstacle(scenario.obstacle_by_id(ego_id))
ego_scenario.add_objects(obstacle)
# show_lane(ego_scenario)

In [95]:
ego_scenario.remove_obstacle(ego_scenario.dynamic_obstacles[0])

In [121]:
size = [400, 30]
duration = [0, 0]
tra = ego_scenario.dynamic_obstacles[0].prediction.trajectory
duration[0] = tra.initial_time_step + 40
duration[1] =  tra.initial_time_step + 80
pos = tra.state_list[39].position + [100, 0]
ego_data = plt_window(sce=ego_scenario, dur=duration, draw_params=draw_params, size=size, pos=pos)

fps: 0.0
fps: 7.140273911970242
fps: 9.450642896815427
fps: 9.751013041480261
fps: 11.434657942714988
fps: 13.102024700243405
fps: 13.860282571974452
fps: 14.87311705943142
fps: 15.690199012419573
fps: 15.96962158978993
fps: 16.033247655584624
fps: 15.879575363833235
fps: 15.507455227656038
fps: 15.00723499860045
fps: 14.58147138125564
fps: 14.069664727485137
fps: 14.014270950081746
fps: 13.832844581607054
fps: 13.751132593952978
fps: 13.626010343841775
fps: 13.615814495538823
fps: 13.66898299671592
fps: 13.551506546380088
fps: 13.770936696139758
fps: 13.851467483692103
fps: 13.868181806158429
fps: 14.044373262916508
fps: 13.953234338648759
fps: 13.93199096939495
fps: 13.688527267090146
fps: 13.513468349395032
fps: 13.625886112515081
fps: 13.589411783764518
fps: 13.564688383765773
fps: 13.595332413035687
fps: 13.75084900350336
fps: 13.843555796944726
fps: 13.96110555147129
fps: 13.950581485930515
fps: 13.989428512638629


In [167]:
obsereved = plt_window(sce=scenario, dur=duration, draw_params=draw_params, size=size, pos=pos)

fps: 0.0
fps: 5.241458254448481
fps: 6.976298272518305
fps: 7.436739986512946
fps: 8.052655249325754
fps: 8.010658713375951
fps: 8.260089271032014
fps: 8.203916738431998
fps: 8.456903841195535
fps: 8.596321880751473
fps: 8.73895443092674
fps: 8.605929629138437
fps: 8.570304161765538
fps: 8.6703834572131
fps: 8.785721349913018
fps: 8.790618207710144
fps: 8.786764873399573
fps: 8.736685784414151
fps: 8.782266254192654
fps: 8.861169505980516
fps: 8.96716404380807
fps: 9.056906802182409
fps: 9.13643156856451
fps: 9.136451155796351
fps: 9.17680701330876
fps: 9.204014467644507
fps: 9.251751587723017
fps: 9.234649663578399
fps: 9.216557712319716
fps: 9.16590658892043
fps: 9.124736373913326
fps: 9.174675232670184
fps: 9.13436868819888
fps: 9.036746458641089
fps: 9.06544592447484
fps: 9.057731619471545
fps: 9.093727550990419
fps: 9.076490168170006
fps: 9.058498248358049
fps: 9.097803728691238


In [115]:
plt_data(obsereved)

In [120]:
print(ego_scenario.dynamic_obstacles[0])
print(scenario.dynamic_obstacles[0])

Dynamic Obstacle:

id: 115
initial state: 
position= [  4.85 139.82]
orientation= -0.0
velocity= 30.0
time_step= 24010

Dynamic Obstacle:

id: 116
initial state: 
position= [  4.85 136.62]
orientation= -0.0
velocity= 30.0
time_step= 24034



In [118]:
obstacle = scenario.dynamic_obstacles[0]
scenario.remove_obstacle(obstacle)
ego_scenario.add_objects(obstacle)

In [119]:
ego_scenario.remove_obstacle(ego_scenario.dynamic_obstacles[0])

In [164]:
from commonroad.prediction.prediction import Prediction, Occupancy
def occupancy_at_time(self, time_step: int) -> Union[None, Occupancy]:
    """
    Returns the predicted occupancy of the obstacle at a specific time step.

    :param time_step: discrete time step
    :return: predicted occupancy of the obstacle at time step
    """
    occupancy = None
    if time_step == self.initial_state.time_step:
        shape = self.obstacle_shape.rotate_translate_local(
            self.initial_state.position, self.initial_state.orientation)
        occupancy = Occupancy(time_step, shape)
    elif time_step > self.initial_state.time_step and time_step < (self.initial_state.time_step + len(self._prediction.trajectory.state_list)) and   self._prediction is not None:
        occupancy = self._prediction.occupancy_at_time_step(time_step)
    return occupancy

In [165]:
from commonroad.scenario.obstacle import StaticObstacle, DynamicObstacle, Obstacle
DynamicObstacle.occupancy_at_time = occupancy_at_time

In [166]:
ego_scenario.dynamic_obstacles[0].occupancy_at_time(24500)